# Multilayer Tidal Calculations
In this cookbook we will explore how we can use TidalPy's multilayer tidal functions to calculate tidal heating as a function of radius.
Here we are using a propagation matrix technique (SVC16) that is only valid in the incompressible limit.

**References**:
- SVC16 : Sabadini, Vermeerson, & Cambiotti (2016, DOI: [10.1007/978-94-017-7552-6](https://doi.org/10.1007/978-94-017-7552-6).
- HH14  : Henning & Hurford (2014, DOI: [10.1088/0004-637X/789/1/30](https://doi.org/10.1088/0004-637X/789/1/30)).
- TB05  : Tobie et al. (2005), DOI: [10.1016/j.icarus.2005.04.006](https://doi.org/10.1016/j.icarus.2005.04.006).
- ID    : [IcyDwarf Code](https://github.com/MarcNeveu/IcyDwarf/blob/master/IcyDwarf/Thermal.h) written by Marc Neveu

## Build the planet

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
%matplotlib ipympl
np.seterr(divide='raise')

from TidalPy import build_world
from TidalPy.constants import G
from TidalPy.tools.conversions import orbital_motion2semi_a
from TidalPy.utilities.numpy_helper import find_nearest
from TidalPy.rheology.complex_compliance.compliance_models import maxwell_array, sundberg_array
# Load TidalPy's multilayer functions
from TidalPy.tides.multilayer import fundamental_matrix_orderl2, propagate, decompose
from TidalPy.tides.potential.synchronous_low_e import tidal_potential
from TidalPy.tides.multilayer.heating import calc_radial_tidal_heating
from TidalPy.tides.multilayer.stress_strain import calculate_strain

In [2]:
# For this example we will use the Io-Jupiter system
io = build_world('Io')
io.paint()
jupiter = build_world('Jupiter')
tidal_host_mass = jupiter.mass
eccentricity = 0.0041
orbital_freq = 2. * np.pi / (86400. * 1.76914)
orbital_period = 2. * np.pi / orbital_freq
semi_major_axis = orbital_motion2semi_a(orbital_freq, tidal_host_mass, io.mass)

# We won't be using TidalPy's OOP approach (which currently does not use the multi-layer approach,
#    instead relying on a homogenous model broken up by layers), instead we will pull out a few parameters needed
#    for the multilayer calculations.
radius_array = io.radii
volume_array = io.volume_slices
# Depth array skips the first shell since most viscoelastic properties are not defined there.
depth_array = (io.radius - radius_array)[1:]
gravity_array = io.gravities
density_array = io.densities
pressure_array = io.pressures

# We will give the core and mantle a different viscoelastic state, but to do that we need to know what index
#   corresponds to the top of the core.
core_radius_cutoff = find_nearest(radius_array, io.core.radius)
core_radius_cutoff_real = core_radius_cutoff - 1
mantle_N = len(radius_array[core_radius_cutoff:])
core_N = len(radius_array[:core_radius_cutoff])
core_N_real = core_N - 1
total_N = len(radius_array)
total_N_real = total_N - 1

# Determine some global properties
world_radius = radius_array[-1]
world_surf_area = 4. * np.pi * world_radius**2
surface_gravity = gravity_array[-1]
mantle_bulk_density = np.average(density_array[core_radius_cutoff:])

# Setup Io's viscoelastic state - this is not particularly accurate, just meant for demonstration purposes

# All of the viscoelastic properties are not defined at r=0, so we are skipping the inner-most shell. This is done through
#    the "len(radius_array) - 1"

# Tidal dissipation is not too sensitive to bulk modulus so we will keep it constant
bulk_moduli = 200.0e9 * np.ones(len(radius_array) - 1)
bulk_moduli[:core_radius_cutoff_real] = 800.0e9

# Shear modulus and viscosity is much more important, but to keep this example simple we will assume a
#   constant shear and an increasing viscosity towards the surface
shear_moduli = 50.0e9 * np.ones(len(radius_array) - 1)
shear_moduli[:core_radius_cutoff_real] = 80.0e9

viscosities = 1.e50 * np.ones(len(radius_array) - 1)
viscosities[core_radius_cutoff_real:] = np.logspace(23, 14, mantle_N)

# Now we can calculate the complex_compliance of the world. We will just use a Maxwell model here but several
#    others are available.
complex_compliances = sundberg_array(orbital_freq, shear_moduli**(-1), viscosities)
# We will specifically turn off tidal dissipation in the core by setting the imaginary portion of
#    complex compliance = 0
complex_compliances[:core_radius_cutoff_real] = \
    np.real(complex_compliances[:core_radius_cutoff_real]) + 1.e-50j * np.ones(core_N_real)
complex_shears = complex_compliances**(-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# Now let's see what this looks like
fig_visco, ax_visco = plt.subplots()
ax_shear = ax_visco.twiny()
ax_comp = ax_visco.twiny()
ax_comp.spines["top"].set_position(("axes", 1.2))

ax_visco.plot(viscosities, depth_array/1000., c='b', ls=':')
ax_shear.plot(shear_moduli, depth_array/1000., c='r', ls='-.')
ax_comp.plot(np.real(complex_shears), depth_array/1000., c='orange', ls='-')
ax_comp.plot(np.imag(complex_shears), depth_array/1000., c='orange', ls='--')
ax_visco.set_ylim(ax_visco.get_ylim()[::-1])
ax_comp.set_ylim(ax_comp.get_ylim()[::-1])
ax_shear.set_ylim(ax_shear.get_ylim()[::-1])

ax_visco.set(ylabel='Depth [km]', xlabel='Viscosity [Pa s]', xscale='log')
ax_shear.set(xlabel='Shear Modulus [Pa]', xscale='linear')
ax_comp.set(xlabel='Complex Shear (Solid=Real, Dash=Imag) [Pa$^{-1}$]', xscale='log')
for ax, color in zip([ax_visco, ax_shear, ax_comp], ['b', 'r', 'orange']):
    if ax is ax_visco:
        ax.spines['bottom'].set_color(color)
    else:
        ax.spines['top'].set_color(color)
    ax.xaxis.label.set_color(color)

fig_visco.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Find the fundamental matrix; skip the innermost shell as that will be a boundary condition.
Y, Y_inv, derivative_mtx = fundamental_matrix_orderl2(radius_array[1:], complex_shears, density_array[1:], gravity_array[1:])

In [5]:
# Propagate the tidal solution through the world's shells
central_boundary_condition = np.zeros((6, 3), dtype=np.complex128)
# Roberts & Nimmo (2000): Liquid innermost zone.
central_boundary_condition[2, 0] = 1.0 + 0.0j
central_boundary_condition[3, 1] = 1.0 + 0.0j
central_boundary_condition[5, 2] = 1.0 + 0.0j

tidal_y, tidal_y_deriv = propagate(Y, Y_inv, derivative_mtx, central_boundary_condition, world_radius, order_l=2)

In [6]:
# Decompose the tidal solution into useful properties
#    There is a gradient in this function so we lose another shell. Thus the [1:] in viscoelastic properties
radial_sensitivity_to_shear, (k2, h2, l2) = \
    decompose(tidal_y, tidal_y_deriv, radius_array[1:], gravity_array[1:], complex_shears, bulk_moduli, order_l=2)

print('Surface Love & Shida Numbers')
print(f'k_2 = {k2[-1]:.2e}; h_2 = {h2[-1]:.2e}; l_2 = {l2[-1]:.2e}')

# Let's plot these as a function of depth.
fig_love, love_axes = plt.subplots(ncols=2)
fig_love.suptitle('Love and Shida Numbers')

for ax_i, ax in enumerate(love_axes):
    ax_love_h = ax.twiny()
    ax_love_l = ax.twiny()
    ax_love_l.spines["top"].set_position(("axes", 1.2))
    
    if ax_i == 0:
        ax.plot(np.real(k2), depth_array/1000., c='b', ls='-')
        ax_love_h.plot(np.real(h2), depth_array/1000., c='r', ls='-')
        ax_love_l.plot(np.real(l2), depth_array/1000., c='orange', ls='-')
        
        ax.set(ylabel='Depth [km]', xlabel='Re[$k_{2}$]', xscale='linear')
        ax_love_h.set(xlabel='Re[$h_{2}$]', xscale='linear')
        ax_love_l.set(xlabel='Re[$l_{2}$]', xscale='linear')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax_love_h.set_ylim(ax_love_h.get_ylim()[::-1])
        ax_love_l.set_ylim(ax_love_l.get_ylim()[::-1])
    else:
        ax.plot(-np.imag(k2), depth_array/1000., c='b', ls='-')
        ax_love_h.plot(-np.imag(h2), depth_array/1000., c='r', ls='-')
        ax_love_l.plot(-np.imag(l2), depth_array/1000., c='orange', ls='-')

        ax.set(ylabel='Depth [km]', xlabel='-Im[$k_{2}$]', xscale='log')
        ax_love_h.set(xlabel='-Im[$h_{2}$]', xscale='log')
        ax_love_l.set(xlabel='-Im[$l_{2}$]', xscale='log')
        ax.set_ylim(ax.get_ylim()[::-1])
        ax_love_h.set_ylim(ax_love_h.get_ylim()[::-1])
        ax_love_l.set_ylim(ax_love_l.get_ylim()[::-1])

    for ax2, color in zip([ax, ax_love_h, ax_love_l], ['b', 'r', 'orange']):
        if ax2 is ax_visco:
            ax2.spines['bottom'].set_color(color)
        else:
            ax2.spines['top'].set_color(color)
        ax2.xaxis.label.set_color(color)

fig_love.tight_layout()
plt.show()

Surface Love & Shida Numbers
k_2 = 1.82e-01-3.21e-01j; h_2 = 3.31e-01-5.82e-01j; l_2 = 9.92e-02-1.75e-01j


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Calculate Tidal Displacement
Utilizing the simplified tidal potential (low eccentricity, no obliquity, spin-synchronous).

In [7]:
# Define latitude and longitude domain
colatitude = np.linspace(0.1, 179.9, 25)
latitude = colatitude - 90.
longitude = np.linspace(0., 360., 30)
long_mtx, lat_mtx, rad_mtx = np.meshgrid(np.radians(longitude), np.radians(colatitude), radius_array[1:])

displacements_over_time = []

time_domain = np.linspace(0., orbital_period, 50)

for time in time_domain:
    # Calculate the simplified tidal potential
    potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
        tidal_potential(rad_mtx, long_mtx, lat_mtx, orbital_freq, eccentricity, time)

    # Scale by the tidal solutions
    radial_displacement = tidal_y[0, -1] * potential
    polar_displacement = tidal_y[2, -1] * potential_dtheta
    azimuthal_displacement = tidal_y[2, -1] * potential_dphi / np.sin(lat_mtx)
    
    displacements_over_time.append((time / orbital_period, radial_displacement, polar_displacement, azimuthal_displacement))

In [8]:
# Plot results
fig_disp, axes_disp = plt.subplots(ncols=3, figsize=(8, 5))
plt.subplots_adjust(wspace=.5)
ax_disp_r = axes_disp[0]
ax_disp_th = axes_disp[1]
ax_disp_phi = axes_disp[2]

def update_fig(time_i=0, radius_idx=0):
    time_orb_per, u_r, u_th, u_phi = displacements_over_time[time_i]
    ax_disp_r.clear()
    ax_disp_th.clear()
    ax_disp_phi.clear()
    fig_disp.suptitle(f'{time_orb_per:0.2f} Orbital Period')
    ax_disp_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Radial Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_disp_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Polar Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_disp_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', title='Azimuthal Displacement', xlim=(0, 360), ylim=(-90, 90))
    ax_disp_r.contourf(longitude, latitude, u_r[:, :, radius_idx], 20)
    ax_disp_th.contourf(longitude, latitude, u_th[:, :, radius_idx], 20)
    ax_disp_phi.contourf(longitude, latitude, u_phi[:, :, radius_idx], 20)
    fig_disp.tight_layout()

interact(update_fig, time_i=(0, len(displacements_over_time)-1, 1),
         radius_idx=widgets.IntSlider(min=0, max=len(radius_array[1:])-1, step=1, value=len(radius_array[1:])-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=49), IntSlider(value=97, description='radiu…

<function __main__.update_fig(time_i=0, radius_idx=0)>

## Orbit-Averaged, Tidal Heating as Function of Depth
Here we will use a modified version of the heating(radius) equations found in TB05. 

In [9]:
# We will use the radial sensitivity to shear calculated in a previous step.
vol_heating_as_func_radius = \
    calc_radial_tidal_heating(eccentricity, orbital_freq, semi_major_axis,
                              tidal_host_mass,
                              radius_array[1:], radial_sensitivity_to_shear,
                              complex_shears, order_l=2)

print(f'Total Heating {sum(vol_heating_as_func_radius * volume_array[1:] ):0.2e} W')
# Plot result
fig_heat_depth_mu, ax_heat_depth_mu = plt.subplots()
ax_heat_depth_mu.set(ylabel='Depth [km]', xlabel='Volumetric Heating Rate [W m$^{-3}$]', xscale='log')
ax_heat_depth_mu.plot(vol_heating_as_func_radius, depth_array / 1000.)
ax_heat_depth_mu.set_ylim(ax_heat_depth_mu.get_ylim()[::-1])
fig_heat_depth_mu.tight_layout()
plt.show()

Total Heating 1.23e+15 W


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Strain and Stress Tensors
Here we will calculate the strain and stress tensors using the methods described in TB05 and B13

In [10]:
# Use the co-lat. and long. domain as setup in the displacements step

strain_tensor_over_time = list()

for time in time_domain:
    # Calculate the simplified tidal potential
    potential_STR, potential_dtheta_STR, potential_dphi_STR, potential_d2theta_STR, potential_d2phi_STR, potential_dtheta_dphi_STR = \
        tidal_potential(rad_mtx, long_mtx, lat_mtx, orbital_freq, eccentricity, time)
    
    potential_STR = [potential_STR[:, :, i] for i in range(len(radius_array[1:]))]
    potential_dtheta_STR = [potential_dtheta_STR[:, :, i] for i in range(len(radius_array[1:]))]
    potential_dphi_STR = [potential_dphi_STR[:, :, i] for i in range(len(radius_array[1:]))]
    potential_d2theta_STR = [potential_d2theta_STR[:, :, i] for i in range(len(radius_array[1:]))]
    potential_d2phi_STR = [potential_d2phi_STR[:, :, i] for i in range(len(radius_array[1:]))]
    potential_dtheta_dphi_STR = [potential_dtheta_dphi_STR[:, :, i] for i in range(len(radius_array[1:]))]

    # Use the tidal solutions to scale the potential and find the strains
    strain_tensor = \
        calculate_strain(potential_STR, potential_dtheta_STR, potential_dphi_STR,
                         potential_d2theta_STR, potential_d2phi_STR,
                         potential_dtheta_dphi_STR, tidal_y, tidal_y_deriv,
                         colatitude=lat_mtx[:, :, 0], radius_array=radius_array[1:], shear_moduli=complex_shears)
    
    strain_tensor_over_time.append((time / (60. * 60), strain_tensor))
    # Note, strain tensor flips the radius position (radius comes first)

### Strain over time

In [11]:
# Plot results
fig_strain_time, axes_strain_time = plt.subplots(ncols=3, nrows=2, figsize=(8, 5))
plt.subplots_adjust(wspace=.5, hspace=0.3)
ax_strain_r = axes_strain_time[0, 0]
ax_strain_th = axes_strain_time[0, 1]
ax_strain_phi = axes_strain_time[0, 2]
ax_strain_rth = axes_strain_time[1, 0]
ax_strain_rphi = axes_strain_time[1, 1]
ax_strain_thphi = axes_strain_time[1, 2]


def update_fig(time_i=0, radius_idx=0):
    time_orb_per, strains_by_r = strain_tensor_over_time[time_i]
    strains_at_surf = strains_by_r[radius_idx, :, :, :, :]
    e_rr = strains_at_surf[0, 0, :, :]
    e_thth = strains_at_surf[1, 1, :, :]
    e_phph = strains_at_surf[2, 2, :, :]
    e_rth = 2. * strains_at_surf[0, 1, :, :]
    e_rph = 2. * strains_at_surf[0, 2, :, :]
    e_thph = 2. * strains_at_surf[1, 2, :, :]
    
    ax_strain_r.clear()
    ax_strain_th.clear()
    ax_strain_phi.clear()
    ax_strain_rth.clear()
    ax_strain_rphi.clear()
    ax_strain_thphi.clear()
    fig_strain_time.suptitle(f'Surface Strains; {time_orb_per:0.2f} Orbital Period')
    ax_strain_r.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                    title='$\\epsilon_{rr}$', xlim=(0, 360), ylim=(-90, 90))
    ax_strain_th.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                     title='$\\epsilon_{\\theta\\theta}$', xlim=(0, 360), ylim=(-90, 90))
    ax_strain_phi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                      title='$\\epsilon_{\\phi\\phi}$', xlim=(0, 360), ylim=(-90, 90))
    ax_strain_rth.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                      title='$\\epsilon_{r\\theta}$', xlim=(0, 360), ylim=(-90, 90))
    ax_strain_rphi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                       title='$\\epsilon_{r\\phi}$', xlim=(0, 360), ylim=(-90, 90))
    ax_strain_thphi.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]',
                        title='$\\epsilon_{\\theta\\phi}$', xlim=(0, 360), ylim=(-90, 90))
    ax_strain_r.contourf(longitude, latitude, e_rr, 10)
    ax_strain_th.contourf(longitude, latitude, e_thth, 10)
    ax_strain_phi.contourf(longitude, latitude, e_phph, 10)
    ax_strain_rth.contourf(longitude, latitude, e_rth, 10)
    ax_strain_rphi.contourf(longitude, latitude, e_rph, 10)
    ax_strain_thphi.contourf(longitude, latitude, e_thph, 10)
    
    fig_strain_time.tight_layout()

interact(update_fig, time_i=(0, len(strain_tensor_over_time)-1, 1),
         radius_idx=widgets.IntSlider(min=0, max=len(radius_array[1:])-1, step=1, value=len(radius_array[1:])-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=49), IntSlider(value=97, description='radiu…

<function __main__.update_fig(time_i=0, radius_idx=0)>

### Stress over depth
**Latitude: 0, Longitude: 0**

In [18]:
latitude_idx = find_nearest(latitude, 0.)
longitude_idx = find_nearest(longitude, 0.)

# Pull out data at t=0 and the desired lat/long
time_orbital_period, strains_by_r = strain_tensor_over_time[0]



# Plot results
fig_strain_depth, axes_strain_depth = plt.subplots(ncols=3, nrows=2, figsize=(8, 5))
plt.subplots_adjust(wspace=.6, hspace=0.3)
ax_strain_depth_r = axes_strain_depth[0, 0]
ax_strain_depth_th = axes_strain_depth[0, 1]
ax_strain_depth_phi = axes_strain_depth[0, 2]
ax_strain_depth_rth = axes_strain_depth[1, 0]
ax_strain_depth_rphi = axes_strain_depth[1, 1]
ax_strain_depth_thphi = axes_strain_depth[1, 2]
ax_strain_depth_r.set(ylabel='Depth [km]', title='$\\epsilon_{rr}$')
ax_strain_depth_th.set(title='$\\epsilon_{\\theta\\theta}$')
ax_strain_depth_phi.set(title='$\\epsilon_{\\phi\\phi}$')
ax_strain_depth_rth.set(ylabel='Depth [km]', title='$\\epsilon_{r\\theta}$')
ax_strain_depth_rphi.set(title='$\\epsilon_{r\\phi}$')
ax_strain_depth_thphi.set(title='$\\epsilon_{\\theta\\phi}$')

strains_as_radius = strains_by_r[:, :, :, longitude_idx, latitude_idx]

e_rr_depth = strains_as_radius[:, 0, 0]
e_thth_depth = strains_as_radius[:, 1, 1]
e_phph_depth = strains_as_radius[:, 2, 2]
e_rth_depth = 2. * strains_as_radius[:, 0, 1]
e_rph_depth = 2. * strains_as_radius[:, 0, 2]
e_thph_depth = 2. * strains_as_radius[:, 1, 2]

ax_strain_depth_r.plot(e_rr_depth, depth_array / 1000., 'k-')
ax_strain_depth_th.plot(e_thth_depth, depth_array / 1000., 'k-')
ax_strain_depth_phi.plot(e_phph_depth, depth_array / 1000., 'k-')
ax_strain_depth_rth.plot(e_rth_depth, depth_array / 1000., 'k-')
ax_strain_depth_rphi.plot(e_rph_depth, depth_array / 1000., 'k-')
ax_strain_depth_thphi.plot(e_thph_depth, depth_array / 1000., 'k-')

ax_strain_depth_r.set_ylim(ax_strain_depth_r.get_ylim()[::-1])
ax_strain_depth_th.set_ylim(ax_strain_depth_th.get_ylim()[::-1])
ax_strain_depth_phi.set_ylim(ax_strain_depth_phi.get_ylim()[::-1])
ax_strain_depth_rth.set_ylim(ax_strain_depth_rth.get_ylim()[::-1])
ax_strain_depth_rphi.set_ylim(ax_strain_depth_rphi.get_ylim()[::-1])
ax_strain_depth_thphi.set_ylim(ax_strain_depth_thphi.get_ylim()[::-1])

fig_strain_depth.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Tidal Heating Rate

In [13]:
fig_heat, axes_heat = plt.subplots(ncols=2, figsize=(8, 4))
plt.subplots_adjust(wspace=4, hspace=0.3)

axis_heat = axes_heat[0]
axis_heat_depth = axes_heat[1]

cb_heat = None


heating1_over_time = list()
heating2_over_time = list()

def calc_heating_at_depth(r_index_to_use, time_i_to_use):

    time_orbital_period, strains_by_r = strain_tensor_over_time[time_i_to_use]
    strains_at_r = strains_by_r[r_index_to_use, :, :, :, :]
    # Per Appendix A of Beuthe (2013), the heating rate can be found from the strain as:
    averaged_strain_sqrd = np.zeros_like(strains_at_r[0, 0, :, :])
    stress = np.zeros_like(strains_at_r)
    heating_opt2 = np.zeros_like(strains_at_r[0, 0, :, :])

    trace = strains_at_r[0, 0, :, :] + strains_at_r[1, 1, :, :] + strains_at_r[2, 2, :, :]

    for i in range(3):
        for j in range(3):
            averaged_strain_sqrd += strains_at_r[i, j, :, :] * np.conj(strains_at_r[i, j, :, :])

            stress[i, j, :, :] = 2. * complex_shears[r_index_to_use] * strains_at_r[i, j, :, :]

            if i == j:
                stress[i, j, :, :] += (bulk_moduli[r_index_to_use] - (2. / 3.) * complex_shears[r_index_to_use]) * trace

            heating_opt2 = (orbital_freq / 2.) * (np.imag(stress[i, j, :, :]) * np.real(strains_at_r[i, j, :, :]) -
                                               np.real(stress[i, j, :, :]) * np.imag(strains_at_r[i, j, :, :])) 

    heating_opt1 = orbital_freq * np.imag(complex_shears[r_index_to_use]) * \
        (averaged_strain_sqrd - (1. / 3.) * np.abs(trace)**2) * volume_array[1:][r_index_to_use]

    heating_opt2 *= volume_array[1:][r_index_to_use]

    return heating_opt1, heating_opt2

for ti, t in enumerate(time_domain):    
    heating1_at_depth = list()
    heating2_at_depth = list()
    for ri, r in enumerate(radius_array[1:]):
        heating, heating_2 = calc_heating_at_depth(ri, ti)
        heating1_at_depth.append(heating)
        heating2_at_depth.append(heating_2)

    heating1_at_depth = np.asarray(heating1_at_depth)
    heating2_at_depth = np.asarray(heating2_at_depth)

    heating1_over_time.append(heating1_at_depth)
    heating2_over_time.append(heating2_at_depth)

heating1_over_time = np.asarray(heating1_over_time)
heating2_over_time = np.asarray(heating2_over_time)

def update_fig(time_i=0, r_index =-1, sum_depth=False, fix_negatives = False, show_flux = True, heat_method_2 = True,
               fixed_long=0., fixed_lat=0., orbit_average=False):
    
    global cb_heat
    
    if orbit_average:
        heating = np.sum(heating1_over_time, axis=0)
        heating_2 = np.sum(heating2_over_time, axis=0)
        
        heating1_at_depth = np.sum(heating1_over_time, axis=0)
        heating2_at_depth = np.sum(heating2_over_time, axis=0)
    else:
        heating = heating1_over_time[time_i, :, :, :]
        heating_2 = heating2_over_time[time_i, :, :, :]
        
        heating1_at_depth = heating1_over_time[time_i, :, :, :]
        heating2_at_depth = heating2_over_time[time_i, :, :, :]

    if sum_depth:
        heating = np.sum(heating1_at_depth, axis=0)
        heating_2 = np.sum(heating2_at_depth, axis=0)
    else:
        heating = heating1_at_depth[r_index, :, :]
        heating_2 = heating2_at_depth[r_index, :, :]
    
    if heat_method_2:
        heating_to_use = heating_2
        heating_depth = heating2_at_depth
    else:
        heating_to_use = heating
        heating_depth = heating1_at_depth
    
    if fix_negatives:
        # Check for negatives
        heating_to_use[heating_to_use < 0.] = 0.
        heating_depth[heating_depth < 0.] = 0.
    
    axis_heat.clear()
    axis_heat_depth.clear()
    
    plt.subplots_adjust(wspace=4, hspace=0.3)
    if cb_heat is not None:
        cb_heat.remove()
    axis_heat.set(title='Tidal Heating [W]')
    if show_flux:
        if sum_depth:
            heating_to_use /= world_surf_area
        else:
            heating_to_use /= 2. * np.pi * radius_array[1:][r_index]**2
        axis_heat.set(title='Surface Heating Flux [W m-2]')
    axis_heat.set(ylabel='Latitude [$\deg$]', xlabel='Longitude [$\deg$]', xlim=(0, 360), ylim=(-90, 90))
    cb_data = axis_heat.contourf(longitude, latitude, heating_to_use, 5)
    cb_heat = plt.colorbar(cb_data, ax=axis_heat)
    
    fig_heat.tight_layout()
    
    # plot depth at sub-stellar point
    lat_indx = find_nearest(latitude, fixed_lat)    
    long_indx = find_nearest(longitude, fixed_long)
    print(long_indx, lat_indx)
    print(heating_depth.shape)
    axis_heat_depth.plot(heating_depth[:, lat_indx, long_indx] / volume_array[1:], depth_array/1000.)
    axis_heat_depth.set(xlabel='Tidal Heating [W m-3]', ylabel='Depth [km]')
    axis_heat_depth.set_ylim(axis_heat_depth.get_ylim()[::-1])
    
    axis_heat.plot([longitude[long_indx]], [latitude[lat_indx]], marker='x', c='r')

interact(update_fig, time_i=(0, len(strain_tensor_over_time)-1, 1),
         r_index=widgets.IntSlider(min=0, max=len(radius_array[1:])-1, step=1, value=len(radius_array[1:])-1),
         sum_depth=False, show_flux=True, heat_method_2=True,
         fixed_long=widgets.IntSlider(min=0, max=360, step=30, value=0),
         fixed_lat=widgets.IntSlider(min=-90, max=90, step=30, value=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='time_i', max=49), IntSlider(value=97, description='r_ind…

<function __main__.update_fig(time_i=0, r_index=-1, sum_depth=False, fix_negatives=False, show_flux=True, heat_method_2=True, fixed_long=0.0, fixed_lat=0.0, orbit_average=False)>

## Performance
Here we are just doing a simple check on how the propagation functions perform

In [14]:
run_performance = False

if run_performance:
    print('Building Matricies')
    %timeit fundamental_matrix_orderl2(radius_array[1:], complex_shears[1:], density_array[1:], gravity_array[1:])

    print('Propagating Solutions')
    %timeit propagate(Y, Y_inv, central_boundary_condition, world_radius, order_l=2)

    print('Decomposition')
    %timeit decompose(tidal_y, radius_array[1:], gravity_array[1:], complex_shears[2:], bulk_moduli[2:], order_l=2)